In [23]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

parent_folder = '/home/myp7435/downsampled_videos/1217'
label_paths = [os.path.join(parent_folder, 'labeled', 'cam_'+str(i)+'_labeled.csv') for i in range(4)]
infer_paths = [os.path.join(parent_folder, 'experiment', 'cam_'+str(i)+'_trimmed_output.csv') for i in range(4)]

joints = ['Wrist', 'CMC_thumb', 'MCP_thumb', 'MCP1', 'MCP2', 'MCP3', 'MCP4',
          'IP_thumb', 'PIP1', 'PIP2', 'PIP3', 'PIP4', 'Dip1', 'Dip2', 'Dip3', 'Dip4',
          'Tip_thumb', 'Tip1', 'Tip2', 'Tip3', 'Tip4']

likelihood_cutoffs = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
err_likelihood_cut = {cutoff:[] for cutoff in likelihood_cutoffs}
reject_likelihood_cut = {cutoff:0 for cutoff in likelihood_cutoffs}

for label_path, infer_path in zip(label_paths, infer_paths):
    df_label = pd.read_csv(label_path, header=[2,3], index_col=0)
    df_infer = pd.read_csv(infer_path, header=[0,1], index_col=0)
    
    indices = [int(re.findall(r'\d+', file)[-1]) for file in df_label.index]
    
    
    for i, index in enumerate(tqdm(indices)):
        for joint in joints:
            for likelihood_cutoff in likelihood_cutoffs:
                if df_infer[joint]['likelihood'][i] > likelihood_cutoff:
                    label = np.array([df_label[joint]['x'][i], df_label[joint]['y'][i]])
                    infer = np.array([df_infer[joint]['x'].iloc[index], df_infer[joint]['y'].iloc[index]])
                    dist = np.linalg.norm(label - infer, axis=0)
                    err_likelihood_cut[likelihood_cutoff].append(dist)
                else:
                    reject_likelihood_cut[likelihood_cutoff] += 1

100%|██████████| 126/126 [00:24<00:00,  5.12it/s]


In [30]:
for likelihood_cutoff in likelihood_cutoffs:
    prop_nan = reject_likelihood_cut[likelihood_cutoff]/(len(indices)*len(joints)*len(label_paths))
    err_mean = np.nanmean(err_likelihood_cut[likelihood_cutoff])
    err_std = np.nanstd(err_likelihood_cut[likelihood_cutoff])
    print(f'Test error ({likelihood_cutoff} cutoff): {err_mean:.2f} ± {err_std:.2f} (pixels) | {prop_nan:.3f} rejected')

Test error (0.05 cutoff): 9.19 ± 23.82 (pixels) | 0.075 rejected
Test error (0.1 cutoff): 9.28 ± 24.50 (pixels) | 0.134 rejected
Test error (0.15 cutoff): 9.38 ± 24.79 (pixels) | 0.184 rejected
Test error (0.2 cutoff): 9.58 ± 25.71 (pixels) | 0.247 rejected
Test error (0.25 cutoff): 9.74 ± 26.75 (pixels) | 0.312 rejected
Test error (0.3 cutoff): 9.87 ± 27.60 (pixels) | 0.413 rejected


In [25]:
reject_likelihood_cut[likelihood_cutoff]

4373

In [26]:
likelihood_cutoffs

[0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

In [27]:
reject_likelihood_cut

{0.05: 797, 0.1: 1420, 0.15: 1951, 0.2: 2615, 0.25: 3301, 0.3: 4373}